In [1]:
# define tools

import random


def get_random_number_between(a: int, b: int) -> int:
    """get random integer between given two integers

    Args:
        a (int): random integer 1
        b (int): random integer 2

    Returns:
        int: random integer between a and b
    """
    return int(random.randrange(min(a, b), max(a, b)))

In [2]:
# args
messages = [
    {"role": "system", "content": "You're a genius assistant."},
    {"role": "user", "content": "Give me random number"}
]

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_random_number_between",
            "description": "useful when you need a random number",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {
                        "type": "integer",
                        "description": "an integer from/to which the random number is to be."
                    },
                    "b": {
                        "type": "integer",
                        "description": "an integer from/to which the random number is to be."
                    }
                }
                
            },
            "required": ["a", "b"]
        }
    },
]

In [3]:
# Ask LLM which tool to use and what are the args
import dotenv
from openai import OpenAI


client = OpenAI()
dotenv.load_dotenv("../.env")

response = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=messages,
    tools=tools,
    tool_choice="auto"
)

In [4]:
response.choices[0].message.tool_calls

[ChatCompletionMessageToolCall(id='call_nok2w15Vpcb4JCaZMqn1yLi1', function=Function(arguments='{"a":1,"b":100}', name='get_random_number_between'), type='function')]

In [5]:
# execute the function
import json


available_tools = {
    "get_random_number_between": get_random_number_between
}

messages.append(response.choices[0].message)

for tool in response.choices[0].message.tool_calls:
    # execute function
    function_name = tool.function.name
    function_args = tool.function.arguments
    function_args_json = json.loads(function_args)
    function_response = available_tools[function_name](**function_args_json)

    # add function response to messages
    messages.append(
        {
            "tool_call_id": tool.id,
            "role": "tool",
            "name": function_name,
            "content": str(function_response)
        }
    )


In [6]:
# final answer
second_response = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=messages,
)

In [7]:
second_response

ChatCompletion(id='chatcmpl-8jyt6LOPdWydaTE065zfNQJxt0S1r', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Your random number is 92.', role='assistant', function_call=None, tool_calls=None), logprobs=None)], created=1705969504, model='gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint='fp_aaa20cc2ba', usage=CompletionUsage(completion_tokens=7, prompt_tokens=52, total_tokens=59))

In [8]:
messages

[{'role': 'system', 'content': "You're a genius assistant."},
 {'role': 'user', 'content': 'Give me random number'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_nok2w15Vpcb4JCaZMqn1yLi1', function=Function(arguments='{"a":1,"b":100}', name='get_random_number_between'), type='function')]),
 {'tool_call_id': 'call_nok2w15Vpcb4JCaZMqn1yLi1',
  'role': 'tool',
  'name': 'get_random_number_between',
  'content': '92'}]